In [1]:
import json
import os
import pandas as pd
import pyshark
import random
import re
from collections import OrderedDict
from json import JSONDecoder
from json import loads
from json import dumps
from os.path import isfile, join
from os import listdir
from pprint import pprint
from subprocess import Popen, PIPE, check_output

# Extract funtion is used to flatten nested dictionaries to a Lvl 1 Dictionary
# Cons: It does not avoid collisions
def extract(dict_in, dict_out):
    for key, value in dict_in.iteritems():
        if isinstance(value, dict): # If value itself is dictionary
            extract(value, dict_out)
        elif isinstance(value, unicode):
            # Write to dict_out
            dict_out[key] = value
    return dict_out

# Parse Object Pairs function identifies all keys that are equal at the same level
# and calls make_unique to rename the keyname
def parse_object_pairs(pairs):
    dct = OrderedDict()
    #dct = dict()
    for key, value in pairs:
        if key in dct:
            key = make_unique(key, dct)
        dct[key] = value
    return dct

# Make Unique function is in charge of renaming keys which have the same name 
def make_unique(key, dct):
    counter = 0
    unique_key = key

    while unique_key in dct:
        counter += 1
        unique_key = '{}-{}'.format(key, str(counter).zfill(2))
    return unique_key


# Parse Object Pairs 2 function identifies all keys that are equal at the same level
# and calls make_unique to rename the keyname
# In addition, it will tag the -00 suffix to each label making data more homogeneous
def parse_object_pairs2(pairs):
    dct = OrderedDict()
    dummy_dct = OrderedDict()
    #print("PAIRS:")
    #print(pairs)
    #print("\n")
    for key, value in pairs:
        #print("Make Unique Loop")
        key = make_unique2(key, dct, dummy_dct)
        dct[key] = value
    return dct

# Make Unique 2 function is in charge of renaming keys which have the same name 
def make_unique2(key, dct, dummy_dct):
    counter = 0
    unique_key = key
    #print("Counter %s" %counter)
    #print("Unique Key:")
    #print(unique_key)
    #print("Dictionary")
    #print(dct)
    if unique_key not in dummy_dct:
        #print("*** IF Loop ***")
        #print(dct)
        #print("Pre-Unique: %s" %unique_key)
        unique_key = '{}-{}'.format(key, str(counter).zfill(2))
        #print("Pos-Unique: %s" %unique_key)
        #print("*** Dummy DCT ***")
        dummy_dct[key] = ""
        #print(dummy_dct)
    else:
        while unique_key in dct or unique_key in dummy_dct:
            #print("*** Else Loop ***")
            #print(dct)
            #print("Pre-Unique: %s" %unique_key)
            unique_key = '{}-{}'.format(key, str(counter).zfill(2))
            #print("Pos-Unique: %s" %unique_key)
            counter += 1
    #print("\n")
    return unique_key

# Flatten DIct function is in charge of flattening the nested dictionaries
# While flattening, it will rename the keys at the last level by prepending predecesor key names
def flatten_dict(d, prefix='>'):
    def items():
        # A clojure for recursively extracting dict like values
        for key, value in d.items():
            if isinstance(value, dict):
                for sub_key, sub_value in flatten_dict(value).items():
                    # Key name should imply nested origin of the dict,
                    # so we use a default prefix of __ instead of _ or .
                    if key == "_source-00" or key == "layers-00":
                        yield sub_key, sub_value
                    else:
                        yield key + prefix + sub_key, sub_value
            else:
                yield key, value
    return dict(items())

# Define the Main Directory containing all uncompressed PCAPS
pcaps_dir='/home/cc/Contagio'
# Create a list of the Sub-directories within the Main Directory
directories = [x[0] for x in os.walk(pcaps_dir)]

# Print list of Sub-directories
print(directories)
uncpx_pcaps_dir = directories

# Define required empty lists
failed_pcaps=[]

# Define variables
mw_id_counter = 0

# Create List of Unique IDs to identify to assign to each Malware PCAP
#mw_random_ids = sorted(random.sample(range(10000,20000), 1300))
mw_random_ids = list(range(10000, 14000))
#mw_id = mw_random_ids[0]

# Dictionary for matching Malware ID and Malware Name
mw_id_dict = {}

#Create Empty Dataframe
columns = ['mw_id']
df_ = pd.DataFrame(index=[0], columns=columns)

# The following loop will perform the following actions:
# 1) Loop through every Sub-directory
# 2) Loop through every PCAP within each Sub-directory
# 3) Read every PCAP using tshark filtering only frames containing ssl traffic
#    In addtion, the output of all protocol features will be delivered in a nested JSON format
# 4) The extracted dictionary will be parsed using parse_object_pairs and flatten_dict function
for pcap_dir in uncpx_pcaps_dir[0:1]: #uncpx_pcaps_dir[0:1]
    pcap_dir = "/home/cc/Contagio/Exploit_Kits_and_Malware"
    print("\n")
    print(pcap_dir)
    pcap_dir_files = [f for f in listdir(pcap_dir) if isfile(join(pcap_dir, f))]
    print("\n")
    #pcap_dir_files = ["EK_MALWARE_2014-04-29-fake-flash-updater-02_mailware-traffic-analysis.net.pcap"]
    for i_cap in pcap_dir_files:
        # Get a new malware ID
        mw_id = mw_random_ids[mw_id_counter]
        # Save Malware Id and Name to Dictionary
        mw_id_dict[mw_id] = i_cap
        print("ID: %s" %mw_id)
        print("MW: %s" %mw_id_dict[mw_id])
        # Increase Counter
        mw_id_counter = mw_id_counter + 1
        # Print PCAP
        print(i_cap)
        # Executed Tshark Commands to Filter frames containing SSL in PCAP
        command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "ssl", "-T", "fields", "-e", "frame.number"]
        #command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "ssl", "-T", "json"]
        #command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-2", "-R", "ssl", "-Y", "frame.number==1", "-T", "json" ]
        process = Popen(command, universal_newlines=True, stdout=PIPE, stderr=PIPE)
        output, err = process.communicate(b"input data that is passed to subprocess' stdin")
        rc = process.returncode
        # Print Number of Packets in SSL Frame
        #packet_numbers = [int(x) for x in output[:-1].split('\n')]
        packet_numbers = list(map(int, re.findall(r'\d+', output)))
        print("Number of Frames containing SSL: %s" %len(packet_numbers))
        print("\n")
        # Loop for extracting and parsing data per frame on PCAP
        # Change from parse_object_pairs to parse_object_pairs2 if you desire that unique label to have the -00 suffix
        for p_num in packet_numbers:
            command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "frame.number=={}".format(p_num), "-T", "json" ]
            process2 = Popen(command, universal_newlines=True, stdout=PIPE, stderr=PIPE)
            output2, err2 = process2.communicate(b"input data that is passed to subprocess' stdin")
            decoder = JSONDecoder(object_pairs_hook=parse_object_pairs2)
            user_dict = decoder.decode(output2[2:-4])
            # Flatten received dictionary
            user_dict_final = flatten_dict(user_dict)
            # Add Malware ID to Temp Dataframe
            user_dict_final["mw_id"] = mw_id
            # Create Temp Dataframe
            df = pd.DataFrame(user_dict_final, index=[1])
            # Append Temp Dataframe to Main Dataframe (df_)
            #df_ = pd.concat([df_,df])
            df_ = df_.append([df], ignore_index=True)

print("\nFailed PCAPS")
print(failed_pcaps)


['/home/cc/Contagio', '/home/cc/Contagio/Exploit_Kits_and_Malware', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2014-08-13-sandbox-analysis-pcap-files_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2015-01-08-pcap-files-associated-with-malware-from-82.244.160.22_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploits_and_Scans', '/home/cc/Contagio/APT', '/home/cc/Contagio/GENERAL_DEFCON_17', '/home/cc/Contagio/General', '/home/cc/Contagio/CRIME', '/home/cc/Contagio/CRIME/tbotpcaps']


/home/cc/Contagio/Exploit_Kits_and_Malware


ID: 10000
MW: EK_MALWARE_2014-03-09-callback-traffic-from-physical-host_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-03-09-callback-traffic-from-physical-host_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10001
MW: EK_MALWARE_2014-08-29-phishing-malware-01-sandbox-analysis_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-08-29-phishing-malware-01-sandbox-analysis_mailwa

Number of Frames containing SSL: 0


ID: 10037
MW: EK_MALWARE_2014-09-04-phishing-malware-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-09-04-phishing-malware-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10038
MW: EK_MALWARE_2015-01-22-Angler-EK-failed-VM-infection-attempt-2-of-3_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-01-22-Angler-EK-failed-VM-infection-attempt-2-of-3_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10039
MW: EK_MALWARE_2014-05-28-Angler-EK-uses-java-exploit_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-05-28-Angler-EK-uses-java-exploit_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10040
MW: EK_MALWARE_2014-05-22-Fiesta-EK-traffic-02_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-05-22-Fiesta-EK-traffic-02_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10041
MW: EK_MALWARE_2014-04-07-Nuclear-EK-traffic_mailware-t

Number of Frames containing SSL: 1


ID: 10076
MW: EK_MALWARE_2014-11-05-malwr.com-analysis-of-image001N.exe_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-11-05-malwr.com-analysis-of-image001N.exe_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10077
MW: EK_MALWARE_2014-07-22-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-07-22-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10078
MW: EK_MALWARE_2014-07-03-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-07-03-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 111


ID: 10079
MW: EK_MALWARE_2014-06-08-Infinity-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-06-08-Infinity-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 1


ID: 10080
MW: EK_MALWARE_2014-08-08-phishing-malware-01-sandbox-analysis_mailware-traffic-analysis.net.pcap
EK_MALW

Number of Frames containing SSL: 0


ID: 10114
MW: EK_MALWARE_2014-11-13-phishing-malware-traffic-in-a-VM_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-11-13-phishing-malware-traffic-in-a-VM_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10115
MW: EK_MALWARE_2015-02-05-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-02-05-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10116
MW: EK_MALWARE_2014-05-16-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-05-16-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10117
MW: EK_MALWARE_2014-05-08-Nuclear-EK-traffic-08_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-05-08-Nuclear-EK-traffic-08_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 38


ID: 10118
MW: EK_MALWARE_2014-06-09-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-06-09-Nuclea

Number of Frames containing SSL: 0


ID: 10152
MW: EK_MALWARE_2014-04-28-Angler-EK-traffic-using-flash-exploit_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-04-28-Angler-EK-traffic-using-flash-exploit_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10153
MW: EK_MALWARE_2014-08-01-Magnitude-EK-malware-payload-sandbox-analysis-4-of-4_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-08-01-Magnitude-EK-malware-payload-sandbox-analysis-4-of-4_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10154
MW: EK_MALWARE_2014-09-04-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-09-04-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10155
MW: EK_MALWARE_2014-03-19-Goon-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-03-19-Goon-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10156
MW: EK_MALWARE_2014-05-25-Angler-EK-traff

...

Number of Frames containing SSL: 0


ID: 10343
MW: EK_MALWARE_2015-02-08-traffic-analysis-exercise_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-02-08-traffic-analysis-exercise_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 245


ID: 10344
MW: EK_MALWARE_2014-04-28-Angler-EK-traffic-using-java-exploit_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-04-28-Angler-EK-traffic-using-java-exploit_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10345
MW: EK_MALWARE_2014-11-22-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-11-22-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 144


ID: 10346
MW: EK_MALWARE_2014-02-03-malware-callback-from-physical-host_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-02-03-malware-callback-from-physical-host_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10347
MW: EK_MALWARE_2014-06-03-Angler-EK-malware-payload-san

Number of Frames containing SSL: 0


ID: 10381
MW: EK_MALWARE_2015-01-22-Angler-EK-failed-VM-infection-attempt-1-of-3_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-01-22-Angler-EK-failed-VM-infection-attempt-1-of-3_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10382
MW: EK_MALWARE_2014-09-28-Styx-EK-failed-attempts_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-09-28-Styx-EK-failed-attempts_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10383
MW: EK_MALWARE_2015-01-13-Dyre-phishing-run-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-01-13-Dyre-phishing-run-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 167


ID: 10384
MW: EK_MALWARE_2014-07-16-phishing-email-malware-sandbox-analysis_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-07-16-phishing-email-malware-sandbox-analysis_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10385
MW: EK_MALWARE_2014-1

Number of Frames containing SSL: 0


ID: 10418
MW: EK_MALWARE_2014-09-16-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-09-16-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 3


ID: 10419
MW: EK_MALWARE_2014-04-01-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-04-01-Fiesta-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10420
MW: EK_MALWARE_2014-06-16-FlashPack-EK-traffic-first-run_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-06-16-FlashPack-EK-traffic-first-run_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10421
MW: EK_MALWARE_2014-06-20-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-06-20-Angler-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 25


ID: 10422
MW: EK_MALWARE_2015-02-06-Rig-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-02-06-Rig-EK-traffic_mailwar

Number of Frames containing SSL: 0


ID: 10457
MW: EK_MALWARE_2014-01-14-magnitude-EK_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-01-14-magnitude-EK_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10458
MW: EK_MALWARE_2014-02-13-Goon-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-02-13-Goon-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10459
MW: EK_MALWARE_2014-07-11-Angler-EK-malware-payload-sandbox-analysis_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-07-11-Angler-EK-malware-payload-sandbox-analysis_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10460
MW: EK_MALWARE_2014-09-30-phishing-malware-VM-infection_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-09-30-phishing-malware-VM-infection_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 72


ID: 10461
MW: EK_MALWARE_2014-06-13-Fake-flash-updater-traffic_mailware-traffic-analysis.net.pcap


Number of Frames containing SSL: 22


ID: 10495
MW: EK_MALWARE_2013-09-28_mailware-traffic-analysis.net.pcap
EK_MALWARE_2013-09-28_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10496
MW: EK_MALWARE_Live-analysis-6b20036e7b3ae7a24231ff351a9251e1_mailware-traffic-analysis.net.pcap
EK_MALWARE_Live-analysis-6b20036e7b3ae7a24231ff351a9251e1_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10497
MW: EK_MALWARE_2014-07-02-fake-USPS-notification-sandbox-analysis_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-07-02-fake-USPS-notification-sandbox-analysis_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10498
MW: EK_MALWARE_2015-02-05-Rig-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2015-02-05-Rig-EK-traffic_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10499
MW: EK_MALWARE_2014-11-06-Nuclear-EK-traffic_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-11-06-Nuclear-

In [2]:
# Create list of discovered features
total_features = list(df_.columns.values)
print ("\n Number of Features:")
print(len(total_features))
#pprint(total_features)



 Number of Features:
4035


In [3]:
# Create mw_id_series
mw_id_series = pd.Series(mw_id_dict)
# Save mw_id_series
mw_id_series.to_pickle('mw_id_series.p')
# Save SSL df (df_)
df_.to_pickle('mw_ssl_dataframe.p')
# Save SSL features list
ssl_features_file = open('ssl_features.txt', 'w')
for item in total_features:
    ssl_features_file.write("%s\n" % item)

In [4]:
# Pretty function walks a dictionary and prints it based with specific number of spaces per level
def pretty(d, indent=0):
    for key, value in d.items():
        print('    ' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('    ' * (indent+1) + str(value))

# Pretty function walks a dictionary and prints it based with specific number of spaces per level
def pretty_save(d, out_tree, indent=0):
        for key, value in d.items():
            out_tree.write('    ' * indent + str(key)+"\n")
            if isinstance(value, dict):
                pretty_save(value, out_tree, indent+1)
            else:
                out_tree.write('    ' * (indent+1) + str(value)+"\n")
            
# SortOD function read a dictionary and sorts keys in alphabetical order per level          
def sortOD(od):
    res = OrderedDict()
    for k, v in sorted(od.items()):
        if isinstance(v, dict):
            res[k] = sortOD(v)
        else:
            res[k] = v
    return res

# Variable for creating an Ordered Dictionary for Features
features_tree = OrderedDict()

# Read the list of features and transfer them to an ordered dictionary
for feature_item in total_features:
    t = features_tree
    for part in feature_item.split('>'):
        t = t.setdefault(part, {})

# Sort features in alphabetical order
features_tree_ordered = sortOD(features_tree)

# Print Ordered Dictionary in Pretty Mode
#pretty(features_tree_ordered)

# Save SSL features tree list
with open('ssl_features_tree.txt', 'w') as out_tree:
    pretty_save(features_tree_ordered, out_tree)
out_tree.close()

# Sources:

## Tshark
https://ask.wireshark.org/questions/58343/2-pass-filter-in-wiresharktshark
https://hackertarget.com/tshark-tutorial-and-filter-examples/
https://www.wireshark.org/docs/wsug_html_chunked/AppToolstshark.html
http://www.thegeekstuff.com/2012/07/wireshark-filter/?ref=driverlayer

## Pandas and JSON
https://www.dataquest.io/blog/python-json-tutorial/
https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
https://pandas.pydata.org/pandas-docs/stable/merging.html
https://stackoverflow.com/questions/23019119/converting-multilevel-nested-dictionaries-to-pandas-dataframe
http://pbpython.com/pandas-list-dict.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it
https://pandas.pydata.org/pandas-docs/stable/options.html
http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/
https://www.digitalocean.com/community/tutorials/an-introduction-to-the-pandas-package-and-its-data-structures-in-python-3

## Unique Dictionary Keys
https://stackoverflow.com/questions/29321677/python-json-parser-allow-duplicate-keys
https://stackoverflow.com/questions/36086470/parsing-python-json-with-multiple-same-strings-with-different-values
https://stackoverflow.com/questions/29321677/python-json-parser-allow-duplicate-keys/29323197

## Flattening Dictionaries
https://stackoverflow.com/questions/12507206/python-recommended-way-to-walk-complex-dictionary-structures-imported-from-json
https://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys

## Zip Files
https://unix.stackexchange.com/questions/270555/how-to-get-number-of-files-in-a-7z